In [ ]:
# NOTE: This uses a modified ethereum-etl here: https://github.com/MSilb7/optimism-etl

import sys
sys.path.append("../../helper_functions")
import ethereumetl_utils as eetl
import web3py_utils as wpy
import pandas_utils as pu
sys.path.pop()

import pandas as pd
from dotenv import load_dotenv
import requests
import datetime
load_dotenv()
import os

In [ ]:
configs = [
        # ['base_goerli',os.environ.get('BASE_GOERLI_NODE')]
        ['op_goerli',os.environ.get('OP_GOERLI_TEAM_NODE')]
]

max_workers = 8
start_timestamp = pu.convert_text_timestamp_to_int('2023-05-19 00:00:00')
end_timestamp = pu.convert_text_timestamp_to_int('2023-05-20 00:00:00')

In [ ]:
# Get Transactions
for i in configs:

    folder = i[0]
    uri = i[1]
#     block_arr = wpy.get_blockrange_by_timedelta(i[1],trailing_hours,'hours')
    st_bl = wpy.getBlockByTimestamp(i[1],start_timestamp)# block_arr[0]
    en_bl = wpy.getBlockByTimestamp(i[1],end_timestamp) # block_arr[1]
    print(st_bl)
    print(en_bl)
    # eetl.get_ethereum_etl(st_bl,en_bl,folder,uri, max_workers)

In [ ]:
# Get Receipts
for i in configs:
    folder = i[0]
    uri = i[1]
    eetl.get_eth_etl_receipts(folder, uri)

### Process Results

In [ ]:
opg_df = pd.read_csv(os.getcwd() + '/downloads/op_goerli/transactions.csv')
rec_df = pd.read_csv(os.getcwd() + '/downloads/op_goerli/receipts.csv')
rec_df = rec_df.rename(columns={'transaction_hash':'hash'})
rec_df = rec_df[['hash','l1_gas_used','l1_fee','l1_gas_price','l1_fee_scalar']]

df = opg_df.merge(rec_df,on='hash',how='left')
#REMOVE DSEPOSITS
df = df[df['transaction_type'] != 126]
df['all_calldata_gas'] = df['l1_gas_used']
df['effective_l1_gas_used'] = df['l1_fee'] / df['l1_gas_price']
display(df)